In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from merge_years.import_data import get_full_data

# Data

In [2]:
raw_data = get_full_data('../raw_data')

## Data split

In [3]:
# Filling the Caglar's position

j=0
for i,row in raw_data.iterrows():
    if row['name']=='caglar söyüncü':
        print(j)
        raw_data['position'].at[i]='DEF'
        j+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [4]:
raw_data['selected'].unique() # done

array([     0,  14029,  22804, ..., 275199, 273494, 270474])

In [5]:
# train and test split 
ra = 2

#test_df = raw_data[(raw_data.season > 20) & (raw_data.GW >= 29-ra)]
#train_df = raw_data[~((raw_data.season > 20) & (raw_data.GW >= 29-ra))]
#
#test_df.shape, train_df.shape

In [6]:
test_df = raw_data[(raw_data.season > 20) & (raw_data.GW >= 34-ra)]
train_df = raw_data[~((raw_data.season > 20) & (raw_data.GW >= 34-ra))]

In [7]:
test_df.shape, train_df.shape

((4924, 40), (110068, 40))

In [8]:
#train_df =  train_df[train_df['minutes'] > 0]

#train_df =  train_df[train_df['name'] == ]


## Data preprocessing

In [9]:
# Train and test sets with interesting features

train_df = train_df.drop(columns=['dreamteam_yearly_average', 'team_level', 'opponent_level', 
                                   'team_name', 'team_id', 'fixture', 'element', 'team_a_score', 
                                   'team_h_score', 'transfers_in', 'transfers_out', 'kickoff_time', 
                                   'kickoff_date', 'opponent_team', 'influence', 'creativity', 'threat', 
                                  'transfers_balance']).copy()

test_df = test_df.drop(columns=['dreamteam_yearly_average', 'team_level', 'opponent_level', 
                                 'team_name', 'team_id', 'fixture', 'element', 'team_a_score', 
                                 'team_h_score', 'transfers_in', 'transfers_out', 'kickoff_time', 
                                 'kickoff_date', 'opponent_team', 'influence', 'creativity', 'threat', 
                                'transfers_balance']).copy()

# 27 features left -> drop selected?

In [10]:
#train_df['selected'].unique()

In [11]:
train_df.head()

,name,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,ict_index,minutes,own_goals,...,red_cards,saves,selected,total_points,value,was_home,yellow_cards,GW,season,position
0,aaron connolly,0,0,1,0,1,0,0.0,24,0,...,0,0,0,1,45,False,0,4,20,FWD
1,aaron connolly,0,0,1,0,1,0,2.2,6,0,...,0,0,14029,1,45,True,0,5,20,FWD
2,aaron connolly,0,0,1,0,0,0,2.5,20,0,...,0,0,22804,1,45,False,0,6,20,FWD
3,aaron connolly,0,0,2,0,1,0,0.1,25,0,...,0,0,32699,1,45,False,0,7,20,FWD
4,aaron connolly,0,3,53,1,0,2,20.1,79,0,...,0,0,35026,13,45,True,0,8,20,FWD


### Rolling Average (ra)

In [12]:
# Features to include in ra (keep name)

rolling_train_df = train_df.drop(columns=['was_home', 'GW', 'season', 'position'])
rolling_test_df = test_df.drop(columns=['was_home', 'GW', 'season', 'position'])

# Ra by name

rolling_train_df = rolling_train_df.groupby('name').rolling(ra, closed = 'left').mean()
rolling_test_df = rolling_test_df.groupby('name').rolling(ra, closed = 'left').mean()

# Creating a rolling points column

rolling_train_df.rename(columns={'total_points':'rolling_points'},inplace=True)
rolling_test_df.rename(columns={'total_points':'rolling_points'},inplace=True)

# Reinserting a total points column (not rolled)

total_points_train = train_df['total_points']
total_points_test = test_df['total_points']

result = map(lambda position:position, total_points_train)
rolling_train_df['total_points']=list(result)
result = map(lambda position:position, total_points_test)
rolling_test_df['total_points']=list(result)

# Reset index

train_df_ra = rolling_train_df.reset_index()
test_df_ra = rolling_test_df.reset_index()

In [13]:
rolling_train_df

assists  bonus  bps  clean_sheets  goals_conceded  \
name                                                                       
aaron connolly 0           NaN    NaN  NaN           NaN             NaN   
               1           NaN    NaN  NaN           NaN             NaN   
               2           0.0    0.0  1.0           0.0             1.0   
               3           0.0    0.0  1.0           0.0             0.5   
               4           0.0    0.0  1.5           0.0             0.5   
...                        ...    ...  ...           ...             ...   
ørjan nyland   114979      0.0    0.0  0.0           0.0             0.0   
               114980      0.0    0.0  0.0           0.0             0.0   
               114981      0.0    0.0  0.0           0.0             0.0   
               114982      0.0    0.0  0.0           0.0             0.0   
               114983      0.0    0.0  0.0           0.0             0.0   

                       goals_scored     ict_index  minutes  own_goals  \
name                                                                    
aaron connolly 0                NaN           NaN      NaN        NaN   
               1                NaN           NaN      NaN        NaN   
               2                0.0  1.100000e+00     15.0        0.0   
               3                0.0  2.350000e+00     13.0        0.0   
               4                0.0  1.300000e+00     22.5        0.0   
...                             ...           ...      ...        ...   
ørjan nyland   114979           0.0  1.243450e-14      0.0        0.0   
               114980           0.0  1.243450e-14      0.0        0.0   
               114981           0.0  1.243450e-14      0.0        0.0   
               114982           0.0  1.243450e-14      0.0        0.0   
               114983           0.0  1.243450e-14      0.0        0.0   

                       penalties_missed  penalties_saved  red_cards  saves  \
name                                                                         
aaron connolly 0                    NaN              NaN        NaN    NaN   
               1                    NaN              NaN        NaN    NaN   
               2                    0.0              0.0        0.0    0.0   
               3                    0.0              0.0        0.0    0.0   
               4                    0.0              0.0        0.0    0.0   
...                                 ...              ...        ...    ...   
ørjan nyland   114979               0.0              0.0        0.0    0.0   
               114980               0.0              0.0        0.0    0.0   
               114981               0.0              0.0        0.0    0.0   
               114982               0.0              0.0        0.0    0.0   
               114983               0.0              0.0        0.0    0.0   

                       selected  rolling_points  value  yellow_cards  \
name                                                                   
aaron connolly 0            NaN             NaN    NaN           NaN   
               1            NaN             NaN    NaN           NaN   
               2         7014.5             1.0   45.0           0.0   
               3        18416.5             1.0   45.0           0.0   
               4        27751.5             1.0   45.0           0.0   
...                         ...             ...    ...           ...   
ørjan nyland   114979  302548.0             0.0   40.0           0.0   
               114980  301118.5             0.0   40.0           0.0   
               114981  297836.5             0.0   40.0           0.0   
               114982  292894.0             0.0   40.0           0.0   
               114983  289525.0             0.0   40.0           0.0   

                       total_points  
name                                 
aaron connolly 0                  1  
               1  

In [14]:
# adding to train: was_home, GW, season, position

result = map(lambda gw:gw, train_df['GW'])
train_df_ra['GW'] = list(result)

result_1 = map(lambda season:season, train_df['season'])
train_df_ra['season'] = list(result_1)

result_2 = map(lambda home:home, train_df['was_home'])
train_df_ra['was_home'] = list(result_2)

result_5 = map(lambda position:position, train_df['position'])
train_df_ra['position'] = list(result_5)

# adding test: was_home, GW, season, position 

result = map(lambda gw:gw, test_df['GW'])
test_df_ra['GW'] = list(result)

result_1 = map(lambda season:season, test_df['season'])
test_df_ra['season'] = list(result_1)

result_2 = map(lambda home:home, test_df['was_home'])
test_df_ra['was_home'] = list(result_2)

result_5 = map(lambda position:position, test_df['position'])
test_df_ra['position'] = list(result_5)


In [15]:
# Drop col: level_1 and nans

train_df_ra.drop(columns=['level_1'], inplace=True)
test_df_ra.drop(columns=['level_1'], inplace=True)

train_preproc = train_df_ra.dropna()
test_preproc = test_df_ra.dropna()

In [16]:
train_preproc.head()

,name,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,ict_index,minutes,own_goals,...,saves,selected,rolling_points,value,yellow_cards,total_points,GW,season,was_home,position
2,aaron connolly,0.0,0.0,1.0,0.0,1.0,0.0,1.10,15.0,0.0,...,0.0,7014.5,1.0,45.0,0.0,1,6,20,False,FWD
3,aaron connolly,0.0,0.0,1.0,0.0,0.5,0.0,2.35,13.0,0.0,...,0.0,18416.5,1.0,45.0,0.0,1,7,20,False,FWD
4,aaron connolly,0.0,0.0,1.5,0.0,0.5,0.0,1.30,22.5,0.0,...,0.0,27751.5,1.0,45.0,0.0,13,8,20,True,FWD
5,aaron connolly,0.0,1.5,27.5,0.5,0.5,1.0,10.10,52.0,0.0,...,0.0,33862.5,7.0,45.0,0.0,1,9,20,False,FWD
6,aaron connolly,0.0,1.5,28.0,0.5,0.5,1.0,11.85,62.0,0.0,...,0.0,72985.0,7.0,45.0,0.0,8,10,20,True,FWD


In [17]:
train_preproc.shape

(107096, 23)

### Encoding

In [18]:
from sklearn.preprocessing import LabelEncoder

# Encoding was_home feature
# Train:

encoder = LabelEncoder()
encoder.fit(train_preproc[['was_home']])

train_preproc['was_home'] = encoder.transform(train_preproc[['was_home']])

# Test:

encoder_2 = LabelEncoder()
encoder_2.fit(test_preproc[['was_home']])

test_preproc['was_home'] = encoder_2.transform(test_preproc[['was_home']])

/Users/Stemitsiotis/.pyenv/versions/3.8.6/envs/fflpred/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-18-8730b044da5e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_preproc['was_home'] = encoder.transform(train_preproc[['was_home']])
<ipython-input-18-8730b044da5e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [19]:
from sklearn.preprocessing import OneHotEncoder

# Encoding position
# Train:

ohe = OneHotEncoder(sparse=False)
ohe.fit(train_preproc[['position']])

position_encoded = ohe.transform(train_preproc[['position']])

train_preproc["DEF"],train_preproc["FWD"],train_preproc["GK"],train_preproc["MID"]= position_encoded.T

# Test:

ohe_2 = OneHotEncoder(sparse=False)
ohe_2.fit(test_preproc[['position']])

position_encoded_2 = ohe_2.transform(test_preproc[['position']])

test_preproc["DEF"],test_preproc["FWD"],test_preproc["GK"],test_preproc["MID"]= position_encoded_2.T

<ipython-input-19-5843488ac307>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_preproc["DEF"],train_preproc["FWD"],train_preproc["GK"],train_preproc["MID"]= position_encoded.T
<ipython-input-19-5843488ac307>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_preproc["DEF"],test_preproc["FWD"],test_preproc["GK"],test_preproc["MID"]= position_encoded_2.T


In [20]:
ohe.categories_

[array(['DEF', 'FWD', 'GK', 'MID'], dtype=object)]

In [21]:
train_preproc = train_preproc.drop(columns=['position'])
test_preproc = test_preproc.drop(columns=['position'])

In [22]:
test_preproc.head()

,name,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,ict_index,minutes,own_goals,...,value,yellow_cards,total_points,GW,season,was_home,DEF,FWD,GK,MID
2,aaron connolly,0.0,0.0,2.5,0.0,0.0,0.0,0.5,11.5,0.0,...,52.0,0.0,0,34,21,1,0.0,1.0,0.0,0.0
3,aaron connolly,0.0,0.0,2.5,0.0,0.0,0.0,0.5,11.5,0.0,...,52.0,0.0,0,35,21,0,0.0,1.0,0.0,0.0
4,aaron connolly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,52.0,0.0,0,36,21,1,0.0,1.0,0.0,0.0
5,aaron connolly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,52.0,0.0,0,37,21,1,0.0,1.0,0.0,0.0
6,aaron connolly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,52.0,0.0,1,38,21,0,0.0,1.0,0.0,0.0


## Train and Test

In [23]:
X_train = train_preproc.drop(columns=['total_points', 'name'])
y_train = train_preproc['total_points']

X_test = test_preproc.drop(columns=['total_points', 'name'])
y_test = test_preproc['total_points']

In [24]:
X_train_scale = X_train.drop(columns=['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK'])
X_test_scale = X_test.drop(columns=['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK'])

In [25]:
from sklearn.preprocessing import MinMaxScaler

# MinMax scaling numerical features

scaler = MinMaxScaler()
scaler.fit(X_train_scale)

# Transform X_train

X_train_final = X_train[['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK']].reset_index(drop = True)\
        .join(pd.DataFrame(scaler.transform(X_train_scale),
             columns=X_train_scale.columns))

In [26]:
# Transform X_test

X_test_final = X_test[['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK']].reset_index(drop = True)\
        .join(pd.DataFrame(scaler.transform(X_test_scale),
             columns=X_test_scale.columns))

In [27]:
X_test_final.head()

,GW,season,was_home,FWD,DEF,MID,GK,assists,bonus,bps,...,minutes,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,rolling_points,value,yellow_cards
0,34,21,1,1.0,0.0,0.0,0.0,0.0,0.0,0.111111,...,0.127778,0.0,0.0,0.0,0.0,0.0,0.004402,0.166667,0.142857,0.0
1,35,21,0,1.0,0.0,0.0,0.0,0.0,0.0,0.111111,...,0.127778,0.0,0.0,0.0,0.0,0.0,0.004398,0.166667,0.142857,0.0
2,36,21,1,1.0,0.0,0.0,0.0,0.0,0.0,0.087963,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.004384,0.145833,0.142857,0.0
3,37,21,1,1.0,0.0,0.0,0.0,0.0,0.0,0.087963,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.004377,0.145833,0.142857,0.0
4,38,21,0,1.0,0.0,0.0,0.0,0.0,0.0,0.087963,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.004381,0.145833,0.142857,0.0


## KNN Model

In [28]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

knn_regressor = KNeighborsRegressor(n_neighbors=23).fit(X_train_final, y_train)

knn_score = cross_val_score(knn_regressor, X_train_final, y_train).mean()
knn_score

0.21697509940464554

In [29]:
from sklearn.metrics import mean_absolute_error

y_pred = knn_regressor.predict(X_test_final)

mean_absolute_error(y_test,y_pred)

1.0549088159085556

### Prediction 

In [32]:
for i,value in enumerate(y_pred):
    print(y_test.iloc[i],value)

0 0.13043478260869565
0 0.34782608695652173
0 0.08695652173913043
0 0.21739130434782608
1 0.9130434782608695
2 0.30434782608695654
1 1.391304347826087
3 1.565217391304348
5 2.739130434782609
6 2.0434782608695654
0 0.0
0 0.13043478260869565
0 0.13043478260869565
0 0.08695652173913043
0 0.043478260869565216
2 3.347826086956522
3 3.3043478260869565
10 3.260869565217391
3 3.5217391304347827
9 3.347826086956522
5 1.4782608695652173
0 3.0
3 1.391304347826087
2 2.260869565217391
0 1.826086956521739
0 0.0
0 0.13043478260869565
3 0.13043478260869565
2 2.9565217391304346
3 2.9565217391304346
1 2.5217391304347827
0 0.2608695652173913
0 0.08695652173913043
0 0.0
0 0.043478260869565216
0 0.13043478260869565
0 0.2608695652173913
0 0.08695652173913043
0 0.0
0 0.043478260869565216
0 2.3043478260869565
0 1.826086956521739
1 0.43478260869565216
1 0.5652173913043478
1 1.565217391304348
6 2.0
1 1.2173913043478262
2 1.9565217391304348
6 2.5217391304347827
1 1.391304347826087
2 2.4347826086956523
8 3.173913

0 0.34782608695652173
0 0.34782608695652173
0 0.0
0 0.043478260869565216
0 0.0
0 0.34782608695652173
0 0.34782608695652173
0 0.6086956521739131
0 0.0
0 0.043478260869565216
1 3.869565217391304
5 4.478260869565218
1 2.652173913043478
6 3.0869565217391304
6 0.9130434782608695
2 1.826086956521739
7 1.2173913043478262
2 2.4782608695652173
6 2.782608695652174
6 1.434782608695652
1 2.391304347826087
0 2.4347826086956523
6 0.7391304347826086
1 0.7391304347826086
0 0.0
0 0.34782608695652173
0 0.34782608695652173
1 0.0
7 0.08695652173913043
21 2.347826086956522
2 5.608695652173913
7 4.0
1 3.391304347826087
13 3.0869565217391304
11 0.13043478260869565
0 1.0434782608695652
1 1.826086956521739
1 1.3043478260869565
1 1.3478260869565217
0 0.043478260869565216
0 0.043478260869565216
0 0.0
0 0.043478260869565216
0 0.08695652173913043
0 3.5652173913043477
1 2.782608695652174
5 1.7826086956521738
0 3.3043478260869565
5 2.0434782608695654
3 2.5652173913043477
2 2.260869565217391
1 2.4347826086956523
3 1.

2 2.4782608695652173
7 3.260869565217391
5 2.6956521739130435
7 2.4782608695652173
0 2.391304347826087
6 0.0
0 0.6956521739130435
0 0.5652173913043478
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.2608695652173913
0 0.5652173913043478
0 0.43478260869565216
0 0.34782608695652173
1 0.34782608695652173
1 0.043478260869565216
0 0.0
0 0.0
0 0.0
0 0.0
0 0.2608695652173913
1 2.0434782608695654
2 1.3043478260869565
2 1.8695652173913044
2 2.391304347826087
2 2.0869565217391304
2 2.260869565217391
8 3.217391304347826
1 2.9565217391304346
3 1.9130434782608696
9 2.869565217391304
2 1.7391304347826086
1 2.869565217391304
7 2.869565217391304
1 2.0434782608695654
2 2.739130434782609
5 1.7826086956521738
0 0.13043478260869565
0 0.2608695652173913
0 0.043478260869565216
0 0.0
0 0.0
2 0.782608695652174
1 0.8260869565217391
1 2.652173913043478
0 1.5217391304347827
0 1.0
0 0.08695652173913043
0 0.0
0 0.08695652173913043
0 0.6086956521739131
0 0.30434782608695654
0 0.21739130434782608
0 0.043478260869565216
0 0.

0 0.0
0 0.34782608695652173
0 0.043478260869565216
0 0.0
0 0.043478260869565216
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.2608695652173913
0 0.0
0 0.34782608695652173
0 0.08695652173913043
0 0.0
0 0.043478260869565216
1 3.5652173913043477
0 2.4782608695652173
2 1.9565217391304348
0 2.782608695652174
1 1.6956521739130435
0 0.4782608695652174
1 0.08695652173913043
0 0.13043478260869565
1 0.43478260869565216
0 0.2608695652173913
0 0.4782608695652174
2 2.3043478260869565
2 2.652173913043478
2 2.9565217391304346
6 2.4782608695652173
2 1.9130434782608696
3 3.3043478260869565
1 3.652173913043478
6 2.9130434782608696
2 2.130434782608696
2 3.0869565217391304
0 0.043478260869565216
0 0.043478260869565216
0 0.0
0 0.08695652173913043
0 0.043478260869565216
0 0.0
0 0.0
0 0.0
1 0.0
2 0.0
0 0.0
0 0.34782608695652173
0 0.043478260869565216
0 0.0
0 0.043478260869565216
0 0.0
0 0.08695652173913043
0 0.043478260869565216
0 0.043478260869565216
0 0.043478260869565216
3 0.17391304347826086


In [80]:
# Dataframe with y_pred and actual total points

test_pred_df = test_preproc[['name', 'GW', 'season', 'FWD' ,'DEF', 'MID', 'GK', 'total_points']]
test_pred_df['y_pred'] = y_pred
test_pred_df

<ipython-input-80-bb80fbd7d637>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pred_df['y_pred'] = y_pred


,name,GW,season,FWD,DEF,MID,GK,total_points,y_pred
2,aaron connolly,34,21,1.0,0.0,0.0,0.0,0,0.130435
3,aaron connolly,35,21,1.0,0.0,0.0,0.0,0,0.347826
4,aaron connolly,36,21,1.0,0.0,0.0,0.0,0,0.086957
5,aaron connolly,37,21,1.0,0.0,0.0,0.0,0,0.217391
6,aaron connolly,38,21,1.0,0.0,0.0,0.0,1,0.913043
...,...,...,...,...,...,...,...,...,...
4919,ørjan nyland,35,21,0.0,0.0,0.0,1.0,0,0.000000
4920,ørjan nyland,35,21,0.0,0.0,0.0,1.0,0,0.000000
4921,ørjan nyland,36,21,0.0,0.0,0.0,1.0,0,0.000000
4922,ørjan nyland,37,21,0.0,0.0,0.0,1.0,0,0.000000


In [56]:
# DEF : top actual and predicted for 5 players

def_test_df = test_pred_df[test_pred_df.DEF == 1]
GW_list=[35,36,37,38]

# top predict

top_predicted_def=def_test_df[def_test_df['GW']==34].sort_values(by='y_pred',ascending=False).head(5)

for GW in GW_list:
    df1=def_test_df[def_test_df['GW']==GW].sort_values(by='y_pred',ascending=False).head(5)
    top_predicted_def=pd.concat([top_predicted_def,df1])

# top actual

top_actual_def=def_test_df[def_test_df['GW']==34].sort_values(by='total_points',ascending=False).head(5)

for GW in GW_list:
    df11=def_test_df[def_test_df['GW']==GW].sort_values(by='total_points',ascending=False).head(5)
    top_actual_def=pd.concat([top_actual_def,df11])

top_predicted_def

,name,GW,season,FWD,DEF,MID,GK,total_points,y_pred
948,conor townsend,34,21,0.0,1.0,0.0,0.0,5,3.695652
2997,luke thomas,34,21,0.0,1.0,0.0,0.0,2,3.565217
1344,enda stevens,34,21,0.0,1.0,0.0,0.0,0,3.565217
4762,wesley fofana,34,21,0.0,1.0,0.0,0.0,1,3.565217
1392,ezgjan alioski,34,21,0.0,1.0,0.0,0.0,1,3.565217
1730,harry maguire,35,21,0.0,1.0,0.0,0.0,0,3.521739
4657,trent alexander-arnold,35,21,0.0,1.0,0.0,0.0,7,3.434783
2964,luke ayling,35,21,0.0,1.0,0.0,0.0,2,3.347826
3764,ozan kabak,35,21,0.0,1.0,0.0,0.0,0,3.086957
3674,ola aina,35,21,0.0,1.0,0.0,0.0,0,3.086957


In [66]:
# FWD : top actual and predicted for 3 players

def_test_fwd = test_pred_df[test_pred_df.FWD == 1]
GW_list=[35,36,37,38]

# top predict

top_predicted_fwd = def_test_fwd[def_test_fwd['GW']==34].sort_values(by='y_pred',ascending=False).head(3)

for GW in GW_list:
    df2 = def_test_fwd[def_test_fwd['GW']==GW].sort_values(by='y_pred',ascending=False).head(3)
    top_predicted_fwd=pd.concat([top_predicted_fwd,df2])

top_predicted_fwd

# top actual

top_actual_fwd = def_test_fwd[def_test_fwd['GW']==34].sort_values(by='total_points',ascending=False).head(3)

for GW in GW_list:
    df22 = def_test_fwd[def_test_fwd['GW']==GW].sort_values(by='total_points',ascending=False).head(3)
    top_actual_fwd=pd.concat([top_actual_fwd,df22])

top_actual_fwd

,name,GW,season,FWD,DEF,MID,GK,total_points,y_pred
3391,michail antonio,34,21,1.0,0.0,0.0,0.0,13,0.478261
1083,danny welbeck,34,21,1.0,0.0,0.0,0.0,12,1.956522
1222,dominic calvert-lewin,34,21,1.0,0.0,0.0,0.0,8,1.260870
713,callum wilson,35,21,1.0,0.0,0.0,0.0,13,0.956522
4044,roberto firmino,35,21,1.0,0.0,0.0,0.0,13,1.000000
1063,danny ings,35,21,1.0,0.0,0.0,0.0,13,0.086957
4080,rodrigo moreno,36,21,1.0,0.0,0.0,0.0,12,0.608696
800,che adams,36,21,1.0,0.0,0.0,0.0,9,2.739130
1085,danny welbeck,36,21,1.0,0.0,0.0,0.0,9,2.782609
3809,patrick bamford,37,21,1.0,0.0,0.0,0.0,12,2.695652


In [64]:
# MID : top actual and predicted for 5 players

def_test_mid = test_pred_df[test_pred_df.MID == 1]
GW_list=[35,36,37,38]

# Top predict

top_predicted_mid=def_test_mid[def_test_mid['GW']==34].sort_values(by='y_pred',ascending=False).head(5)

for GW in GW_list:
    df3=def_test_mid[def_test_mid['GW']==GW].sort_values(by='y_pred',ascending=False).head(5)
    top_predicted_mid=pd.concat([top_predicted_mid,df3])

# top actual

top_actual_mid = def_test_mid[def_test_mid['GW']==34].sort_values(by='total_points',ascending=False).head(5)

for GW in GW_list:
    df33 = def_test_mid[def_test_mid['GW']==GW].sort_values(by='total_points',ascending=False).head(5)
    top_actual_mid=pd.concat([top_actual_mid,df33])

top_actual_mid

,name,GW,season,FWD,DEF,MID,GK,total_points,y_pred
1614,gareth bale,34,21,0.0,0.0,1.0,0.0,21,2.347826
2606,kai havertz,34,21,0.0,0.0,1.0,0.0,16,2.086957
1780,heung-min son,34,21,0.0,0.0,1.0,0.0,12,4.173913
3889,pierre-emerick aubameyang,34,21,0.0,0.0,1.0,0.0,12,0.391304
3426,mohamed naser el sayed elneny,34,21,0.0,0.0,1.0,0.0,11,1.130435
1259,eberechi eze,35,21,0.0,0.0,1.0,0.0,14,3.391304
4211,sadio mané,35,21,0.0,0.0,1.0,0.0,11,3.217391
1331,emile smith rowe,35,21,0.0,0.0,1.0,0.0,11,1.260870
3463,morgan gibbs-white,35,21,0.0,0.0,1.0,0.0,10,0.913043
673,bruno miguel borges fernandes,35,21,0.0,0.0,1.0,0.0,10,5.130435


In [81]:
# GK : top actual and predicted for 2 players

def_test_gk = test_pred_df[test_pred_df.GK == 1]
GW_list=[35,36,37,38]

# top predict 

top_predicted_gk=def_test_gk[def_test_gk['GW']==34].sort_values(by='y_pred',ascending=False).head(2)

for GW in GW_list:
    df4=def_test_gk[def_test_gk['GW']==GW].sort_values(by='y_pred',ascending=False).head(2)
    top_predicted_gk=pd.concat([top_predicted_gk,df4])

# top actual

top_actual_gk = def_test_gk[def_test_gk['GW']==34].sort_values(by='total_points',ascending=False).head(2)

for GW in GW_list:
    df44 = def_test_gk[def_test_gk['GW']==GW].sort_values(by='total_points',ascending=False).head(2)
    top_actual_gk=pd.concat([top_actual_gk,df44])

top_actual_gk

,name,GW,season,FWD,DEF,MID,GK,total_points,y_pred
1279,edouard mendy,34,21,0.0,0.0,0.0,1.0,7,1.043478
4231,sam johnstone,34,21,0.0,0.0,0.0,1.0,7,3.739130
1280,edouard mendy,35,21,0.0,0.0,0.0,1.0,11,3.086957
217,alisson ramses becker,35,21,0.0,0.0,0.0,1.0,10,3.608696
25,aaron ramsdale,36,21,0.0,0.0,0.0,1.0,10,3.260870
219,alisson ramses becker,36,21,0.0,0.0,0.0,1.0,10,2.652174
1547,francisco casilla cortés,37,21,0.0,0.0,0.0,1.0,9,0.000000
3110,martin dubravka,37,21,0.0,0.0,0.0,1.0,7,4.434783
1269,ederson santana de moraes,38,21,0.0,0.0,0.0,1.0,12,0.478261
27,aaron ramsdale,38,21,0.0,0.0,0.0,1.0,9,3.347826


In [74]:
print(top_predicted_def.shape)
print(top_predicted_fwd.shape)
print(top_predicted_mid.shape)
print(top_predicted_gk.shape)
print(top_actual_def.shape)
print(top_actual_fwd.shape)
print(top_actual_mid.shape)
print(top_actual_gk.shape)

(25, 9)
(15, 9)
(25, 9)
(10, 9)
(25, 9)
(15, 9)
(25, 9)
(10, 9)


In [82]:
# Top predicted and actual with all positions in one df

top_predited = pd.concat([top_predicted_def, top_predicted_fwd, top_predicted_mid, top_predicted_gk])
top_actual = pd.concat([top_actual_def, top_actual_fwd, top_actual_mid, top_actual_gk])
top_actual

,name,GW,season,FWD,DEF,MID,GK,total_points,y_pred
4301,serge aurier,34,21,0.0,1.0,0.0,0.0,14,2.826087
2437,jonny evans,34,21,0.0,1.0,0.0,0.0,10,3.478261
1801,héctor bellerín,34,21,0.0,1.0,0.0,0.0,8,0.782609
266,andreas christensen,34,21,0.0,1.0,0.0,0.0,8,1.826087
3653,nélson cabral semedo,34,21,0.0,1.0,0.0,0.0,8,3.391304
...,...,...,...,...,...,...,...,...,...
219,alisson ramses becker,36,21,0.0,0.0,0.0,1.0,10,2.652174
1547,francisco casilla cortés,37,21,0.0,0.0,0.0,1.0,9,0.000000
3110,martin dubravka,37,21,0.0,0.0,0.0,1.0,7,4.434783
1269,ederson santana de moraes,38,21,0.0,0.0,0.0,1.0,12,0.478261


In [77]:
top_actual.groupby('GW').sum()

,season,FWD,DEF,MID,GK,total_points,y_pred
GW,,,,,,,
34,315,3.0,5.0,5.0,2.0,167,30.913043
35,315,3.0,5.0,5.0,2.0,168,30.173913
36,315,3.0,5.0,5.0,2.0,172,33.130435
37,315,3.0,5.0,5.0,2.0,156,33.695652
38,315,3.0,5.0,5.0,2.0,186,28.913043


In [78]:
top_predited.groupby('GW').sum()

,season,FWD,DEF,MID,GK,total_points,y_pred
GW,,,,,,,
34,315,3.0,5.0,5.0,2.0,46,53.217391
35,315,3.0,5.0,5.0,2.0,55,63.695652
36,315,3.0,5.0,5.0,2.0,69,53.043478
37,315,3.0,5.0,5.0,2.0,40,59.217391
38,315,3.0,5.0,5.0,2.0,90,50.434783


In [86]:
top_predited['DEF'].sum()/top_actual['DEF'].sum()

1.0

## Pipeline

In [79]:
#from sklearn.pipeline import Pipeline
#
#pipe = Pipeline([
#    ('imputer', SimpleImputer()),
#    ('scaler', StandardScaler())
#])